In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
from flopy.utils.gridintersect import GridIntersect
#import pyemu

from shapely.geometry import Polygon, Point
import shapefile
from shapely.prepared import prep

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams.update({'font.size': 14})

In [ ]:
bins_pth = os.path.join('..', 'bins', 'win') if 'nt' in os.name else os.path.join('..', 'bins', 'linux') # Binaries
shapefile_pth = os.path.join('..', 'data', 'raw_data', 'shapefiles')
observations_pth = os.path.join('..', 'data', 'observations') # Measured data (field obs)

In [ ]:
org_model_ws = os.path.join('..', 'temp_flopy_lumprem')
os.listdir(org_model_ws)

In [ ]:
tmp_model_ws = os.path.join('..', 'temp_ml_param') # Safe to delete
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
os.listdir(tmp_model_ws)

In [ ]:
ml_name = 'hagfors_1'

In [ ]:
sim = flopy.mf6.MFSimulation.load(ml_name, 'mf6', os.path.join(bins_pth, 'mf6'), tmp_model_ws)

Set all data external so that PEST can adjust parameter values during history matching:

In [ ]:
sim.set_all_data_external(True)

In [ ]:
sim.write_simulation()

In [ ]:
gwf = sim.get_model(ml_name)

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Model grid', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

plt.show()

## Generate pilot points
3D-pilot points will be generated for the following areal properties:
- SY & SS (perhaps SS could be skipped?)
- Porosity and possibly diffusion/dispersion and any other parameter needed to represent transport
- Kh and Kz

In addition, pilot points will be generated along the following linear features:
- Streambed hydraulic conductivity along Creek Örbäcken (SFR package)
- GHB conductance

### 3D pilot points
Because we want to do a data-worth analysis, we should consider the total amount of pilot points to be employed to find a suitable compromise between adjustable parameters and model run-time.

For this reason, we will create different sets of pilot points for each parameter type (in order to not use an exessive amount of pps).

Sy, SS and possibly porosity (along with other parameters that govern transport) could be parameterized using a coarser pp-spacing, so let's start with that.

However, before we start, let's import the observation locations, so that we can see how the pilot points will be located in relation to them:

In [ ]:
sfr_obs = pd.read_excel(os.path.join(observations_pth, 'obs_flow_and_stage.xlsx'))
head_obs = pd.read_excel(os.path.join(observations_pth, 'obs_head_per_layer.xlsx')).drop_duplicates(subset=['POINT_X', 'POINT_Y'])
head_obs['TYPE'] = 'HEAD'

In [ ]:
obs_points = pd.concat([head_obs, sfr_obs])
display(obs_points)

#### Coarse 3D pilot points
Let's create the coarse pilot point distribution:

In [ ]:
ml_boundary = shapefile.Reader(os.path.join(shapefile_pth, 'model_boundary.shp')) # Model boundary shapefile
mlb_shape = np.array(np.rint(ml_boundary.shapeRecords()[0].shape.points)) # Model boundary array

Instantiate a shapely polygon of the model boundary:

In [ ]:
mlb_shapely = Polygon(mlb_shape)

Create prepared polygon of the model boundary:

In [ ]:
#mlb_shapely_prep = prep(mlb_shapely.buffer(50)) buffer cause an issue because the grid can't be sampled outside the model boundary
mlb_shapely_prep = prep(mlb_shapely)

Construct a rectangular mesh of points:

In [ ]:
xmin, xmax, ymin, ymax = 426900, 427700, 6654650, 6655350 # These are the same coordinates used to construct the base-grid
resolution = 30 # Equal space (in meters) between pilot points
basepoints = []
for lat in np.arange(xmin, xmax, resolution):
    for lon in np.arange(ymin, ymax, resolution):
        basepoints.append(Point((round(lat,4), round(lon,4))))

Use the shapely `contains` (point-in-polygon method) to select points inside the model boundary (increase the number of pps once workflow is confirmed working):

In [ ]:
basepip = [] # Basepoints in polygon
for point in basepoints:
    if mlb_shapely_prep.contains(point):
        basepip.append(point)
print(f'Number of points per layer: {len(basepip)}') # We need to extend it into three dimensions

Plot the position of the pilot points on top of the model grid:

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Coarse pilot point locations', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

x = np.array([i.coords[0] for i in basepip])[:,0]
y = np.array([i.coords[0] for i in basepip])[:,1]
plt.scatter(x, y, s=5, c='black', alpha=0.5, label='PILOT POINT')

for category in obs_points['TYPE'].unique():
    x = obs_points.loc[obs_points['TYPE'] == category]['POINT_X']
    y = obs_points.loc[obs_points['TYPE'] == category]['POINT_Y']
    ax.scatter(x, y, s=20, alpha=0.6, label=category)
    
plt.legend()
plt.show()

These points now need to be assigned a z-value, since we are going to use 3D-pilot points. To do this, we will need to intersect the model grid and retrieve the z-values of each layer, so that a copy of these pps can be positioned in each of the three layers:

In [ ]:
ix = GridIntersect(gwf.modelgrid)

In [ ]:
pp_intersect = {
    'cellids': [],
    'vertices': [],
    'ixshapes': [],
}
for point in basepip:
    pp_intersect['cellids'].append(ix.intersect(point).cellids[0])
    pp_intersect['vertices'].append(ix.intersect(point).vertices[0])
    pp_intersect['ixshapes'].append(ix.intersect(point).ixshapes[0])

In [ ]:
pp_intersect = pd.DataFrame(pp_intersect)
display(pp_intersect)

We can see that the order is respected:

In [ ]:
[i.coords[0] for i in basepip][:5]

Let's use the cellids to sample pilot point elevations (**this takes about 2 minutes on my laptop for 295 cells** and could/should probably be speed up somehow - considering it has to be done for top, botm1, botm2 and botm3):

In [ ]:
def get_grid_elevation(elevation_array, index_array):
    '''
    returns list of elevations
    '''
    
    elevations = list(elevation_array)
    indices = list(index_array)
    
    return [elevation_array[i] for i in index_array]

In [ ]:
pp_intersect['top'] = get_grid_elevation(elevation_array = gwf.modelgrid.top, index_array=pp_intersect['cellids'])

In [ ]:
pp_intersect['botm_1'] = get_grid_elevation(gwf.modelgrid.botm[0], pp_intersect['cellids'])
pp_intersect['botm_2'] = get_grid_elevation(gwf.modelgrid.botm[1], pp_intersect['cellids'])
pp_intersect['botm_3'] = get_grid_elevation(gwf.modelgrid.botm[2], pp_intersect['cellids'])

In [ ]:
pp_intersect

Create pps in-between the layer boundaries (i.e. vertically centered in the cells):

In [ ]:
pp_intersect['pps_l1'] = (pp_intersect['top'] + pp_intersect['botm_1']) / 2

In [ ]:
pp_intersect['pps_l2'] = (pp_intersect['botm_1'] + pp_intersect['botm_2']) / 2

In [ ]:
pp_intersect['pps_l3'] = (pp_intersect['botm_2'] + pp_intersect['botm_3']) / 2

In [ ]:
display(pp_intersect)

Add x, y and clean up df:

In [ ]:
pp_intersect['x'] = np.array([i.coords[0] for i in basepip])[:,0]
pp_intersect['y'] = np.array([i.coords[0] for i in basepip])[:,1]

In [ ]:
pp_intersect = pp_intersect[['x', 'y', 'pps_l1', 'pps_l2', 'pps_l3']]

In [ ]:
pp_intersect

Visualize

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.set_box_aspect([2,2,1])

x, y = pp_intersect.x.values, pp_intersect.y.values

ax.scatter(x, y, pp_intersect.pps_l1.values)
ax.scatter(x, y, pp_intersect.pps_l2.values)
ax.scatter(x, y, pp_intersect.pps_l3.values)
plt.title('pps in 3d-space')
pyplot.show()

Convert the dataframe into a 3d pilot point file format:

In [ ]:
pp_coarse3d = pd.melt(pp_intersect, id_vars=['x', 'y'], value_vars=['pps_l1', 'pps_l2', 'pps_l3'])

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.set_box_aspect([2,2,1])

x, y = pp_coarse3d.x.values, pp_coarse3d.y.values

ax.scatter(x, y, pp_coarse3d.value.values)
plt.title('pps in 3d-space')
pyplot.show()

Insert columns:

In [ ]:
pp_coarse3d['name'] = [f'ppc{i:04d}' for i in pp_coarse3d.index.values] # ppc = pilot point coarse.

In [ ]:
pp_coarse3d[['zone','val']] = 1, 1
pp_coarse3d['z'] = pp_coarse3d['value']

In [ ]:
pp_coarse3d['layer'] = [int(i[-1]) for i in pp_coarse3d['variable']]

In [ ]:
pp_coarse3d = pp_coarse3d[['name', 'x', 'y', 'z', 'zone', 'val', 'layer']]

In [ ]:
display(pp_coarse3d)

In [ ]:
from lumpyrem import run
#MKPPSTAT
# mkppstat requires no headers in ppoint file ...sigh...
pp_coarse3d.to_csv(os.path.join(tmp_model_ws, 'mkppoints3d_coarse.dat'),
                       header=None, index=False, sep='\t')

In [ ]:
# Input for MKPPSTAT
# A (a)-factor of 1.5 is often reasonable. from tutorial 
npoints_h, npoints_v = 10, 10 # np-horizontal, np-vertical
a_h, a_v = 1.2, 1.2 # a-horizontal, a-vertical
# run MKPPSTAT
run.run_process(
    'mkppstat3d',
    path=tmp_model_ws,
    commands=['mkppoints3d_coarse.dat', npoints_h, a_h, npoints_v, a_v, 'ppstat3d_coarse.dat']
)

In [ ]:
# run PPCOV3D_SVA - pilot point covariance 3d - spatially varying anisotropy
run.run_process(
    'ppcov3d_sva',
    path=tmp_model_ws,
    commands=['ppstat3d_coarse.dat', 'y', 1, 'x',  'cov3d_coarse.mat', '']
)

In [ ]:
# Load cov mat file with Pyemu for further processing
import pyemu
covmat_coarse = pyemu.Cov.from_ascii(os.path.join(tmp_model_ws, "cov3d_coarse.mat"))

# This covaraince matrix can now be used as the base for all pilot point parameters. 
# Note that in this case the variance is 1, so it is easy to scale to a parameters prior varaince
# Depending on how you setup the scrpt, variance can be assigned at various stages (i.e. when running PPCOV_SVA, or by manipulating the matrix later)
# Note that parameter names (headers and row names) come from the parameter name sin the ppoint file. These can have a prefix added by PPCOV_SVA, or changed in the dataframe. The latter is more versatile.
covmat_coarse.to_dataframe().head()

Is this matrix only used during regularization?

In [ ]:
plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(covmat_coarse.as_2d)

In [ ]:
pp_coarse3d[['kh', 'kv', 'sy', 'ss']] = 86.4, 8.64, 0.2, 0.00001

In [ ]:
pp_coarse3d

In [ ]:
pp_coarse3d.to_csv(os.path.join(tmp_model_ws, 'pp3d_coarse.dat'),
                       header=None, index=False, sep='\t')

## Setup parameterization for 3D elements (K, storage, porosity)

In [ ]:
def write_plproc_script(filename, lines):

    with open(filename, 'a') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

Calculate kriging factors (porosity not included yet):

In [ ]:
write_plproc_script(os.path.join(tmp_model_ws, 'plproc1_temp.dat'), [
f'''
### Read model grid ###
cl_mf = read_mf6_grid_specs(file={ml_name}.disv.grb,               &
                            dimensions=2,                             &
                            slist_layer_idomain=idomain1;  layer=1,   &
                            slist_layer_idomain=idomain2;  layer=2,   &
                            slist_layer_idomain=idomain3;  layer=3)



### Read 3D pilot-points file ###
cl_pp = read_list_file(file=pp3d_coarse.dat,               &
                       id_type=character,                  &
                       dimensions=2,                       &
                       slist=zone; col=5,                  &
                       slist=lyr; col=7,                   &
                       plist=kh_pp; col=8,                 &
                       plist=kv_pp; col=9,                 &
                       plist=sy_pp; col=10,                &
                       plist=ss_pp; col=11)


### Calculate kriging factors for each layer ###
calc_kriging_factors_auto_2d(                    &
           target_clist=cl_mf,                   &
           source_clist=cl_pp;select=(lyr==1),   &
           file=factors_pp_lyr1.dat)

calc_kriging_factors_auto_2d(                    &
           target_clist=cl_mf,                   &
           source_clist=cl_pp;select=(lyr==2),   &
           file=factors_pp_lyr2.dat)

calc_kriging_factors_auto_2d(                    &
           target_clist=cl_mf,                   &
           source_clist=cl_pp;select=(lyr==3),   &
           file=factors_pp_lyr3.dat)

'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=tmp_model_ws,
    commands=['plproc1_temp.dat']
)

## Setup parameterization for linear elements (GHBs & SFR)
Setup pilot points for linear boundary conditions General Head Boundaries (GHBs) and Creek Örbäcken (SFR), starting with SFR:

In [ ]:
segfile = pd.read_csv(os.path.join(tmp_model_ws, 'sfr_segfile.dat'), sep='\t', names=['x', 'y', 'seg'])
display(segfile)

Fix the segments. This should probably be done in a better way and perhaps changed after the first round of history matching to get a better representation of stream reaches. This solution is just to make things work.

In [ ]:
def largest_divisor(n):
    a = 1
    for i in range(2, n):
        if n % i == 0:
            a = i
    return a

In [ ]:
nseg = largest_divisor(len(segfile))
#nseg = largest_divisor(nseg) # divide again to reduce the amount of pps from 135 to 45 (perhaps a stupid move)
print(nseg)

In [ ]:
rows_per_seg = int(len(segfile) / nseg)
print(rows_per_seg)

In [ ]:
for i in range(nseg):
    segfile.loc[(i * rows_per_seg):((i + 1) * rows_per_seg), ('seg')] = f's{i + 1}'

In [ ]:
segfile

Rewrite seglist:

In [ ]:
segfile.to_csv(
    os.path.join(tmp_model_ws, 'sfr_segfile.dat'),
    header=None,
    index=False,
    sep='\t',
    float_format='%.3f'
)

### GHB Seglists

In [ ]:
ghb_seglists = []
for filename in os.listdir(tmp_model_ws):
    if 'ghb_' in filename and '_segfile.dat' in filename:
        ghb_seglists.append(filename)

In [ ]:
ghb_seglists_dfs = {}

In [ ]:
for filename in ghb_seglists:
    ghb_seglists_dfs[filename.replace('_segfile.dat', '')] = pd.read_csv(
        os.path.join(tmp_model_ws, filename),
        sep='\t',
        names=['x', 'y', 'seg']
    )

In [ ]:
ghb_seglists_dfs['ghb_red']

Assign segment IDs:

In [ ]:
for k,v in ghb_seglists_dfs.items():
    print(f'rows in {k}: {len(v["seg"])}')

In [ ]:
for k,segfile in ghb_seglists_dfs.items():
    nseg = largest_divisor(len(segfile))
    rows_per_seg = int(len(segfile) / nseg)
    for i in range(nseg):
        segfile.loc[(i * rows_per_seg):((i + 1) * rows_per_seg), ('seg')] = f's{i + 1}'

In [ ]:
ghb_seglists_dfs['ghb_red']

I'm still not sure about the implification of segments... for example there are three rows in ghb_orange and ghb_red each. However, ghb_red is about 5 times the size of ghb_orange... Assigning a single segment to all ghbs except yellow:

In [ ]:
ghb_seglists_dfs

In [ ]:
for k, segfile in ghb_seglists_dfs.items():
    segfile.to_csv(
        os.path.join(tmp_model_ws, f'{k}_segfile.dat'),
        header=None,
        index=False,
        sep='\t',
        float_format='%.3f'
    )

In [ ]:
write_plproc_script(os.path.join(tmp_model_ws, 'plproc_seglist_temp1.dat'), [
f'''
### Read model grid ###
cl_mf = read_mf6_grid_specs(                &
    file={ml_name}.disv.grb,                &
    dimensions=2,                           &
    slist_layer_idomain=idomain1;  layer=1, &
    slist_layer_idomain=idomain2;  layer=2, &
    slist_layer_idomain=idomain3;  layer=3  &
    )


### Read Creek Örbäcken SFR seglist file ###
sl_sfr = read_segfile(file="sfr_segfile.dat", protocol=table)
    
### Read GHB seglist files ###
sl_ghb_red = read_segfile(file="ghb_red_segfile.dat", protocol=table)
sl_ghb_orange = read_segfile(file="ghb_orange_segfile.dat", protocol=table)
sl_ghb_yellow = read_segfile(file="ghb_yellow_segfile.dat", protocol=table)
sl_ghb_limegreen = read_segfile(file="ghb_limegreen_segfile.dat", protocol=table)
sl_ghb_royalblue = read_segfile(file="ghb_royalblue_segfile.dat", protocol=table)
sl_ghb_blueviolet = read_segfile(file="ghb_blueviolet_segfile.dat", protocol=table)
sl_ghb_magenta = read_segfile(file="ghb_magenta_segfile.dat", protocol=table)


### Create clist with sl_sfr as its base ###
cl_sfr_pp = create_clist_from_seglist(seglist=sl_sfr, linkage_type=endpoints, dist_thresh=5.0)

### Create clist with sl_ghb* as its base ###
cl_ghb_red_pp = create_clist_from_seglist(seglist=sl_ghb_red, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_orange_pp = create_clist_from_seglist(seglist=sl_ghb_orange, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_yellow_pp = create_clist_from_seglist(seglist=sl_ghb_yellow, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_limegreen_pp = create_clist_from_seglist(seglist=sl_ghb_limegreen, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_royalblue_pp = create_clist_from_seglist(seglist=sl_ghb_royalblue, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_blueviolet_pp = create_clist_from_seglist(seglist=sl_ghb_blueviolet, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_magenta_pp = create_clist_from_seglist(seglist=sl_ghb_magenta, linkage_type=endpoints, dist_thresh=5.0)


### Write reports (pps will be constructed based on theses reports) ###
cl_sfr_pp.report_dependent_lists(file='report_sfr_seglist.dat')

cl_ghb_red_pp.report_dependent_lists(file='report_ghb_red_seglist.dat')
cl_ghb_orange_pp.report_dependent_lists(file='report_ghb_orange_seglist.dat')
cl_ghb_yellow_pp.report_dependent_lists(file='report_ghb_yellow_seglist.dat')
cl_ghb_limegreen_pp.report_dependent_lists(file='report_ghb_limegreen_seglist.dat')
cl_ghb_royalblue_pp.report_dependent_lists(file='report_ghb_royalblue_seglist.dat')
cl_ghb_blueviolet_pp.report_dependent_lists(file='report_ghb_blueviolet_seglist.dat')
cl_ghb_magenta_pp.report_dependent_lists(file='report_ghb_magenta_seglist.dat')
'''
])

RUN PLPROC:

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=tmp_model_ws,
    commands=['plproc_seglist_temp1.dat']
)

### Create SEGLIST pilot points from the report files

Define a function for writing conductance files:

In [ ]:
def write_conductance(filename, conductance):
    df = pd.read_csv(os.path.join(tmp_model_ws, filename), skiprows=[0, 1, 2])
    
    df['ID'] = df.index + 1
    df['conductance'] = conductance
    df = df[['ID', 'conductance']]
    new_filename = filename.replace('report_', '').replace('_seglist.dat', '')
    new_filename = new_filename + '_cond.dat'
    
    print(f'Writing {new_filename}')
    df.to_csv(
        os.path.join(tmp_model_ws, new_filename),
        header=None,
        index=False,
        sep='\t',
    )

In [ ]:
reportfiles = [i for i in os.listdir(tmp_model_ws) if 'report_' in i]
display(reportfiles)

In [ ]:
for file in reportfiles:
    write_conductance(file, 86.4) # Assigning 86.4 as starting K

### Calculate kriging interpolation factors for the SEGLISTs

In [ ]:
write_plproc_script(os.path.join(tmp_model_ws, 'plproc_seglist_temp2.dat'), [
f'''
### Read model grid ###

cl_mf = read_mf6_grid_specs(  &
    file={ml_name}.disv.grb,  &
    dimensions=3,             & # Note 3D in this case
    slist_layernum = layer,   &
    slist_idomain = idomain   &
    )


### Read Creek Örbäcken SFR seglist file ###
sl_sfr = read_segfile(file="sfr_segfile.dat", protocol=table)
    
### Read GHB seglist files ###
sl_ghb_red = read_segfile(file="ghb_red_segfile.dat", protocol=table)
sl_ghb_orange = read_segfile(file="ghb_orange_segfile.dat", protocol=table)
sl_ghb_yellow = read_segfile(file="ghb_yellow_segfile.dat", protocol=table)
sl_ghb_limegreen = read_segfile(file="ghb_limegreen_segfile.dat", protocol=table)
sl_ghb_royalblue = read_segfile(file="ghb_royalblue_segfile.dat", protocol=table)
sl_ghb_blueviolet = read_segfile(file="ghb_blueviolet_segfile.dat", protocol=table)
sl_ghb_magenta = read_segfile(file="ghb_magenta_segfile.dat", protocol=table)


### Create clist with sl_sfr as its base ###
cl_sfr_pp = create_clist_from_seglist(seglist=sl_sfr, linkage_type=endpoints, dist_thresh=5.0)

### Create clist with sl_ghb* as its base ###
cl_ghb_red_pp = create_clist_from_seglist(seglist=sl_ghb_red, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_orange_pp = create_clist_from_seglist(seglist=sl_ghb_orange, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_yellow_pp = create_clist_from_seglist(seglist=sl_ghb_yellow, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_limegreen_pp = create_clist_from_seglist(seglist=sl_ghb_limegreen, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_royalblue_pp = create_clist_from_seglist(seglist=sl_ghb_royalblue, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_blueviolet_pp = create_clist_from_seglist(seglist=sl_ghb_blueviolet, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_magenta_pp = create_clist_from_seglist(seglist=sl_ghb_magenta, linkage_type=endpoints, dist_thresh=5.0)


### Instruct PLPROC to read the *_cond.dat     ###
### list file to obtain conductance values at  ###
### pilot points by inserting the following    ###
### function into the PLPROC script.           ###
read_list_file(reference_clist='cl_sfr_pp', file='sfr_cond.dat', plist='pp_sfr_cond';column=2)
read_list_file(reference_clist='cl_ghb_red_pp', file='ghb_red_cond.dat', plist='pp_ghb_red_c';column=2)
read_list_file(reference_clist='cl_ghb_orange_pp', file='ghb_orange_cond.dat', plist='pp_ghb_orange_c';column=2)
read_list_file(reference_clist='cl_ghb_yellow_pp', file='ghb_yellow_cond.dat', plist='pp_ghb_yellow_c';column=2)
read_list_file(reference_clist='cl_ghb_limegreen_pp', file='ghb_limegreen_cond.dat', plist='pp_ghb_limegreen_c';column=2)
read_list_file(reference_clist='cl_ghb_royalblue_pp', file='ghb_royalblue_cond.dat', plist='pp_ghb_royalblue_c';column=2)
read_list_file(reference_clist='cl_ghb_blueviolet_pp', file='ghb_blueviolet_cond.dat', plist='pp_ghb_blueviolet_c';column=2)
read_list_file(reference_clist='cl_ghb_magenta_pp', file='ghb_magenta_cond.dat', plist='pp_ghb_magenta_c';column=2)


### Instruct PLPROC to build an SLIST of model ###
### drain cells to which interpolation must    ###
### take place                                 ###
sfr_cells = cl_mf.find_cells_in_lists(file={ml_name}.sfr_packagedata.txt, model_type=mf6_disv, &
    list_col_start=2, keytext_start='top_of_file', keytext_end='end_of_file')

ghb_red_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='top_of_file', keytext_end='2387  ghb_orange')
    
ghb_orange_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2387  ghb_red', keytext_end='2792  ghb_yellow')

ghb_yellow_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2800  ghb_orange', keytext_end='2802  ghb_limegreen')

ghb_limegreen_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='3090  ghb_yellow', keytext_end='1  ghb_royalblue')

ghb_royalblue_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='3057  ghb_limegreen', keytext_end='1  ghb_blueviolet')

ghb_blueviolet_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2802  ghb_royalblue', keytext_end='316  ghb_magenta')

ghb_magenta_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='316  ghb_blueviolet', keytext_end='end_of_file')

### Calculate interpolation factors to these   ###
### model cells through linear interpolation   ###
### along the segments (only run once)         ###
calc_linear_interp_factors(                               &
    source_clist=cl_sfr_pp,                               &
    target_clist=cl_mf;select=(sfr_cells.ne.0),           &
    file="factors_sfr_cells.dat",                         &
    search_radius=50                                      &
    )

calc_linear_interp_factors(                               &
    source_clist=cl_ghb_red_pp,                           &
    target_clist=cl_mf;select=(ghb_red_cells.ne.0),       &
    file="factors_ghb_red_cells.dat",                     &
    search_radius=50                                      &
    )

calc_linear_interp_factors(                               &
    source_clist=cl_ghb_orange_pp,                        &
    target_clist=cl_mf;select=(ghb_orange_cells.ne.0),    &
    file="factors_ghb_orange_cells.dat",                  &
    search_radius=50                                      &
    )

calc_linear_interp_factors(                               &
    source_clist=cl_ghb_yellow_pp,                        &
    target_clist=cl_mf;select=(ghb_yellow_cells.ne.0),    &
    file="factors_ghb_yellow_cells.dat",                  &
    search_radius=50                                      &
    )

calc_linear_interp_factors(                                &
    source_clist=cl_ghb_limegreen_pp,                      &
    target_clist=cl_mf;select=(ghb_limegreen_cells.ne.0),  &
    file="factors_ghb_limegreen_cells.dat",                &
    search_radius=50                                       &
    )

calc_linear_interp_factors(                                &
    source_clist=cl_ghb_royalblue_pp,                      &
    target_clist=cl_mf;select=(ghb_royalblue_cells.ne.0),  &
    file="factors_ghb_royalblue_cells.dat",                &
    search_radius=50                                       &
    )

calc_linear_interp_factors(                                &
    source_clist=cl_ghb_blueviolet_pp,                     &
    target_clist=cl_mf;select=(ghb_blueviolet_cells.ne.0), &
    file="factors_ghb_blueviolet_cells.dat",               &
    search_radius=50                                       &
    )

calc_linear_interp_factors(                                &
    source_clist=cl_ghb_magenta_pp,                        &
    target_clist=cl_mf;select=(ghb_magenta_cells.ne.0),    &
    file="factors_ghb_magenta_cells.dat",                  &
    search_radius=50                                       &
    )
'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=tmp_model_ws,
    commands=['plproc_seglist_temp2.dat']
)

In [ ]:
os.listdir(tmp_model_ws)

### Create final PLPROC script for use in history-matching
(transport pertinent parameters to be added once worflow is OK)

In [ ]:
# Write generic template
with open(os.path.join(tmp_model_ws, 'gen_mf_array.tpl'), 'a') as f:
    f.write('$#p prop_mf.write_in_sequence(format="(1x,1pg18.11)")')

In [ ]:
disv = gwf.get_package('disv')

In [ ]:
ncpl = disv.ncpl.data

In [ ]:
write_plproc_script(os.path.join(tmp_model_ws, 'plproc1.dat'), [
f'''
### Read model grid ###
cl_mf = read_mf6_grid_specs(file={ml_name}.disv.grb,                  &
                            dimensions=2,                             &
                            slist_layer_idomain=idomain1;  layer=1,   &
                            slist_layer_idomain=idomain2;  layer=2,   &
                            slist_layer_idomain=idomain3;  layer=3)

###### --- 3D pps


### Read 3D pilot-points file ###
cl_pp = read_list_file(file=pp3d_coarse.dat,               &
                       id_type=character,                  &
                       dimensions=2,                       &
                       slist=zone; col=5,                  &
                       slist=lyr; col=7,                   &
                       plist=kh_pp; col=8,                 &
                       plist=kv_pp; col=9,                 &
                       plist=sy_pp; col=10,                &
                       plist=ss_pp; col=11)


### Write  ###
prop_mf=new_plist(reference_clist=cl_mf,value=1.0)


###   Horizontal K   ###
### Write kh layer 1 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k_layer1.txt)

### Write kh layer 2 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k_layer2.txt)

### Write kh layer 3 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k_layer3.txt)


###    Vertical K    ###
### Write kv layer 1 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k33_layer1.txt)

### Write kv layer 2 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k33_layer2.txt)

### Write kv layer 3 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.npf_k33_layer3.txt)



###      STORAGE     ###
###        sy        ###
### Write sy layer 1 ###
prop_mf=0.2
prop_mf=sy_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.sto_sy_layer1.txt)


###        ss        ###
### Write ss layer 2 ###
prop_mf=0.000001
prop_mf=ss_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.sto_ss_layer2.txt)

### Write ss layer 3 ###
prop_mf=0.000001
prop_mf=ss_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, model_input_file={ml_name}.sto_ss_layer3.txt)



###### --- SFR SEGLISTS

### Read Creek Örbäcken SFR seglist file ###
sl_sfr = read_segfile(file="sfr_segfile.dat", protocol=table)


### Create clist with sl_sfr as its base ###
cl_sfr_pp = create_clist_from_seglist(seglist=sl_sfr, linkage_type=endpoints, dist_thresh=5.0)


### Instruct PLPROC to read the draincond.dat  ###
### list file to obtain conductance values at  ###
### pilot points by inserting the following    ###
### function into the PLPROC script.           ###
read_list_file(reference_clist='cl_sfr_pp', file='sfr_cond.dat', plist='pp_sfr_cond';column=2)


### Instruct PLPROC to build an SLIST of model ###
### drain cells to which interpolation must    ###
### take place                                 ###


sfr_cells = cl_mf.find_cells_in_lists(file={ml_name}.sfr_packagedata.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=2, keytext_start='top_of_file', &
    keytext_end='end_of_file')


sfr_cond=new_plist(reference_clist=cl_mf,value=0.0)


sfr_cond=pp_sfr_cond.interp_using_file(file=factors_sfr_cells.dat, transform=log)


replace_cells_in_lists(                            &
    old_file={ml_name}.sfr_packagedata.txt,        &
    new_file=new_{ml_name}.sfr_packagedata.txt,    &
    model_type=undefined;nlay=1;ncpl={ncpl},       &
    list_col_start=2,                              &
    keytext_start='top_of_file',                   &
    keytext_end='bottom_of_file',                  &
    plist=sfr_cond;column=9;action='replace'    &
    )



###### --- GHB SEGLISTS

sl_ghb_red = read_segfile(file="ghb_red_segfile.dat", protocol=table)
sl_ghb_orange = read_segfile(file="ghb_orange_segfile.dat", protocol=table)
sl_ghb_yellow = read_segfile(file="ghb_yellow_segfile.dat", protocol=table)
sl_ghb_limegreen = read_segfile(file="ghb_limegreen_segfile.dat", protocol=table)
sl_ghb_royalblue = read_segfile(file="ghb_royalblue_segfile.dat", protocol=table)
sl_ghb_blueviolet = read_segfile(file="ghb_blueviolet_segfile.dat", protocol=table)
sl_ghb_magenta = read_segfile(file="ghb_magenta_segfile.dat", protocol=table)

cl_ghb_red_pp = create_clist_from_seglist(seglist=sl_ghb_red, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_orange_pp = create_clist_from_seglist(seglist=sl_ghb_orange, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_yellow_pp = create_clist_from_seglist(seglist=sl_ghb_yellow, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_limegreen_pp = create_clist_from_seglist(seglist=sl_ghb_limegreen, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_royalblue_pp = create_clist_from_seglist(seglist=sl_ghb_royalblue, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_blueviolet_pp = create_clist_from_seglist(seglist=sl_ghb_blueviolet, linkage_type=endpoints, dist_thresh=5.0)
cl_ghb_magenta_pp = create_clist_from_seglist(seglist=sl_ghb_magenta, linkage_type=endpoints, dist_thresh=5.0)

read_list_file(reference_clist='cl_ghb_red_pp', file='ghb_red_cond.dat', plist='pp_ghb_red_c';column=2)
read_list_file(reference_clist='cl_ghb_orange_pp', file='ghb_orange_cond.dat', plist='pp_ghb_orange_c';column=2)
read_list_file(reference_clist='cl_ghb_yellow_pp', file='ghb_yellow_cond.dat', plist='pp_ghb_yellow_c';column=2)
read_list_file(reference_clist='cl_ghb_limegreen_pp', file='ghb_limegreen_cond.dat', plist='pp_ghb_limegreen_c';column=2)
read_list_file(reference_clist='cl_ghb_royalblue_pp', file='ghb_royalblue_cond.dat', plist='pp_ghb_royalblue_c';column=2)
read_list_file(reference_clist='cl_ghb_blueviolet_pp', file='ghb_blueviolet_cond.dat', plist='pp_ghb_blueviolet_c';column=2)
read_list_file(reference_clist='cl_ghb_magenta_pp', file='ghb_magenta_cond.dat', plist='pp_ghb_magenta_c';column=2)

ghb_red_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='top_of_file', &
    keytext_end='2387  ghb_orange')

ghb_orange_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='2387  ghb_red', &
    keytext_end='2792  ghb_yellow')
    
ghb_yellow_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='2800  ghb_orange', &
    keytext_end='2802  ghb_limegreen')

ghb_limegreen_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='3090  ghb_yellow', &
    keytext_end='1  ghb_royalblue')

ghb_royalblue_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='3057  ghb_limegreen', &
    keytext_end='1  ghb_blueviolet')

ghb_blueviolet_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='2802  ghb_royalblue', &
    keytext_end='316  ghb_magenta')

ghb_magenta_cells = cl_mf.find_cells_in_lists(file={ml_name}.ghb_stress_period_data_1.txt, &
    model_type=undefined;nlay=1;ncpl={ncpl}, list_col_start=1, keytext_start='316  ghb_blueviolet', &
    keytext_end='end_of_file')


ghb_cond=new_plist(reference_clist=cl_mf,value=0.0)


## --- GHB Red
ghb_cond=pp_ghb_red_c.interp_using_file(file=factors_ghb_red_cells.dat, transform=log)

replace_cells_in_lists(old_file={ml_name}.ghb_stress_period_data_1.txt,                                &
    new_file=partial1_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='top_of_file', keytext_end='2387  ghb_orange',                     &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Orange
ghb_cond=pp_ghb_orange_c.interp_using_file(file=factors_ghb_orange_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial1_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=partial2_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='2387  ghb_red', keytext_end='2792  ghb_yellow',                   &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Yellow
ghb_cond=pp_ghb_yellow_c.interp_using_file(file=factors_ghb_yellow_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial2_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=partial3_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='2800  ghb_orange', keytext_end='2802  ghb_limegreen',             &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Limegreen
ghb_cond=pp_ghb_limegreen_c.interp_using_file(file=factors_ghb_limegreen_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial3_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=partial4_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='3090  ghb_yellow', keytext_end='1  ghb_royalblue',                &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Royalblue
ghb_cond=pp_ghb_royalblue_c.interp_using_file(file=factors_ghb_royalblue_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial4_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=partial5_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='3057  ghb_limegreen', keytext_end='1  ghb_blueviolet',            &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Blueviolet
ghb_cond=pp_ghb_blueviolet_c.interp_using_file(file=factors_ghb_blueviolet_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial5_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=partial6_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl}, &
    list_col_start=1, keytext_start='2802  ghb_royalblue', keytext_end='316  ghb_magenta',             &
    plist=ghb_cond;column=4;action='replace')


## --- GHB Magenta
ghb_cond=pp_ghb_magenta_c.interp_using_file(file=factors_ghb_magenta_cells.dat, transform=log)

replace_cells_in_lists(old_file=partial6_{ml_name}.ghb_stress_period_data_1.txt,                       &
    new_file=new_{ml_name}.ghb_stress_period_data_1.txt, model_type=undefined;nlay=1;ncpl={ncpl},      &
    list_col_start=1, keytext_start='316  ghb_blueviolet', keytext_end='end_of_file',                  &
    plist=ghb_cond;column=4;action='replace')

### Write reports ###
report_all_entities(file=report1.dat)
cl_mf.report_dependent_lists(file='report2.dat')
#cl_sfrpp.report_dependent_lists(file='report3.dat')
'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=tmp_model_ws,
    commands=['plproc1.dat']
)

Phew... is the parameterization complete? I think so (at least up until transport parameters)...

The workspace is a bit of a mess, so let's clean up non pertinent files:

In [ ]:
len(os.listdir(tmp_model_ws))

In [ ]:
rm_files = [i for i in os.listdir(tmp_model_ws) if 'temp' in i or 'partial' in i or 'report' in i]

In [ ]:
for file in rm_files:
    os.remove(os.path.join(tmp_model_ws, file))

In [ ]:
len(os.listdir(tmp_model_ws))

What's left? ... make MF read the new SFR and GHB files:

In [ ]:
# Read in the file
with open(os.path.join(tmp_model_ws, f'{ml_name}.ghb'), 'r') as file :
    filedata = file.read()

# Replace the target string
filedata = filedata.replace(f'{ml_name}.ghb_stress_period_data_1.txt', f'new_{ml_name}.ghb_stress_period_data_1.txt')

# Write the file out again
with open(os.path.join(tmp_model_ws, f'{ml_name}.ghb'), 'w') as file:
    file.write(filedata)

In [ ]:
# Read in the file
with open(os.path.join(tmp_model_ws, f'{ml_name}.sfr'), 'r') as file :
    filedata = file.read()

# Replace the target string
filedata = filedata.replace(f'{ml_name}.sfr_packagedata.txt', f'new_{ml_name}.sfr_packagedata.txt')

# Write the file out again
with open(os.path.join(tmp_model_ws, f'{ml_name}.sfr'), 'w') as file:
    file.write(filedata)

In [ ]:
sim.run_simulation()

In [ ]:
sim.get_model().npf.k.plot(colorbar=True, figsize=(10,10))

In [ ]:
sim.get_model().npf.k33.plot(colorbar=True, figsize=(10,10))

In [ ]:
sim.get_model().sto.sy.plot(colorbar=True, figsize=(10,10))

In [ ]:
sim.get_model().sto.ss.plot(colorbar=True, figsize=(10,10))

In [ ]:
##### 

write_plproc_script(os.path.join(tmp_model_ws, 'plproc1.dat'), [
'''
### Read model grid ###
cl_mf = read_mf6_grid_specs(file=experimental.disv.grb,    &
                            dimensions=3,                  &
                            slist_idomain = idomain,       &
                            slist_layernum = layer,        &
                            plist_bottom = bottom) # Is bottom needed?

### Read pilot-points file ###
cl_pp = read_list_file(file=pp3d_coarse.dat,               &
                       id_type=character,                  &
                       dimensions=3,                       &
                       slist=zone; col=5,                  &
                       slist=lyr; col=7,                   &
                       plist=kh_pp; col=8,                 &
                       plist=kv_pp; col=9,                 &
                       plist=sy_pp; col=10,                &
                       plist=ss_pp; col=11)


### Create CLIST partitions so that the function "write_model_input_file" writes ###
### a MODFLOW readable parameter file with the same length as                    ### 
### experimental.npf_k_layer1.txt                                                ###
### Is this the proper way to go about it?                                       ###
cl_mf_layer1 = cl_mf.partition_by_eqn(select=(layer==1))
cl_mf_layer2 = cl_mf.partition_by_eqn(select=(layer==2))
cl_mf_layer3 = cl_mf.partition_by_eqn(select=(layer==3))


### Calculate kriging factors for each CLIST partition ###
### If creating partitions is wrong, then this will be ###
### wrong as well...                                   ###
calc_kriging_factors_3d(target_clist=cl_mf_layer1,                   &
                        source_clist=cl_pp,                          &
                        file=ppfactors1.dat,                         &
                        variogram=exponential,                       &
                        a_hmax=32, a_hmin=28, a_vert=8,              &
                        ang1=60, ang2=20, ang3=0,                    &
                        kriging=ordinary,                            &
                        search_rad_max_hdir=130,                     &
                        search_rad_min_hdir=66,                      &
                        search_rad_vert=33,                          &
                        min_points=1,max_points=10)

calc_kriging_factors_3d(target_clist=cl_mf_layer2,                   &
                        source_clist=cl_pp,                          &
                        file=ppfactors2.dat,                         &
                        variogram=exponential,                       &
                        a_hmax=32, a_hmin=28, a_vert=8,              &
                        ang1=60, ang2=20, ang3=0,                    &
                        kriging=ordinary,                            &
                        search_rad_max_hdir=130,                     &
                        search_rad_min_hdir=66,                      &
                        search_rad_vert=33,                          &
                        min_points=1,max_points=10)

calc_kriging_factors_3d(target_clist=cl_mf_layer3,                   &
                        source_clist=cl_pp,                          &
                        file=ppfactors3.dat,                         &
                        variogram=exponential,                       &
                        a_hmax=32, a_hmin=28, a_vert=8,              &
                        ang1=60, ang2=20, ang3=0,                    &
                        kriging=ordinary,                            &
                        search_rad_max_hdir=130,                     &
                        search_rad_min_hdir=66,                      &
                        search_rad_vert=33,                          &
                        min_points=1,max_points=10)

### Write  ###
prop_mf_l1=new_plist(reference_clist=cl_mf_layer1, value=1.0)
prop_mf_l2=new_plist(reference_clist=cl_mf_layer2, value=1.0)
prop_mf_l3=new_plist(reference_clist=cl_mf_layer3, value=1.0)

### Write kh layer 1 ###
prop_mf_l1=86.4
prop_mf_l1=kh_pp.krige_using_file(file='ppfactors1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array_l1.tpl, model_input_file=experimental.npf_k_layer1.txt)

### Write kh layer 2 ###
prop_mf_l2=1.0E-2
prop_mf_l2=kh_pp.krige_using_file(file='ppfactors2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array_l2.tpl, model_input_file=experimental.npf_k_layer2.txt)

### Write kh layer 3 ###
prop_mf_l3=63.2
prop_mf_l3=kh_pp.krige_using_file(file='ppfactors3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array_l3.tpl, model_input_file=experimental.npf_k_layer3.txt)

### Write reports ###
report_all_entities(file=report1.dat)
cl_mf.report_dependent_lists(file='report2.dat')
'''
])